<h1>Coding Session #1 - Lineare Regression</h1>

Diese Datei ist ein Jupyter Notebook. Dieses besteht aus Textblöcken im Markdown Format und ausführbaren Code-Zellen. Diese erkennen Sie an dem kleinen Pfeilsymbol links daneben.

## 1 Motivation

Künstliche Neuronale Netze bestehen aus einer Vielzahl an Neuronen. Jedes Neuron ist in der Lage, eine simple mathematische Repräsentation zu lernen. Durch Verknüpfung mehrerer Neuronen können komplexe mathematische Funktionen nachgebildet werden. So kommen neuronale Netze in bspw. in der Bildverarbeitung, der Spracherkennung, für Wetter-, Energiebedarfs- oder Verschleißsprognosen zum Einsatz.

## 2 Ziel

Das Grundprinzip eines jeden Neurons ist **lineare Regression**. In diesem Coding Beispiel wird vorerst die Funktionsweise eines einzelnen Neurons betrachtet, bevor mehrere Neuronen zu einem neuronalen Netz verknüpft werden, um dessen Funktionsweise zu verdeutlichen.

## 3 Vorbereitung

Bei dieser Datei (`*.ipynb`) handelt sich um ein Jupyter Notebook. Das ist eine interaktive Datei, in der neben strukturierten Textzellen (`markdown`) Codezellen direkt integriert und ausgeführt werden können. Die Verwendung von `Visual Studio Code` wird empfohlen.

### 3.1 Jupyter Extension installieren
1. Klicken Sie auf Extensions<br>
    <img src="images/vscode_extensions.png" width="300px"></img>
2. Suchen Sie nach "Jupyter" und installieren Sie die Erweiterung.<br>
    <img src="images/install_jupyter.png" width="500px"></img>

### 3.2 Virtuelle Umgebung erstellen

**Virtuelle Umgebungen** sorgen dafür, dass Paketinstallationen nur im lokalen Projektkontext (eben im virtuellen Environment) erfolgen. So werden Versionskonflikte mit bestehenden Installationen vermieden.

**1.** Führen Sie im Terminal folgende Codezelle aus, um eine virtuelle Umgebung zu erstellen:

In [1]:
!python3.12 -m venv .venv

**2.** Führen Sie je nach Betriebssystem eine der folgenden Zellen aus:

Windows:

In [ ]:
!.\.venv\Scripts\activate.bat

Linux & MacOS:

In [2]:
!source .venv/bin/activate

**3.1** Klicken Sie oben rechts in Visual Studio Code auf die Pathon Version<br>
**3.2** Klicken Sie auf _Anderen Kernel auswählen..._
   
<img src="images/prepare001.png" width="800px"></img>

**4.** Klicken Sie auf _Python Umgebungen..._
   
<img src="images/prepare002.png" width="500px"></img>

**3.** Wählen Sie _.venv (Python 3.12.0)_

<img src="images/prepare003.png" width="500px"></img>

### 3.2 Requirements installieren

Zur Vorbereitung stellen Sie bitte sicher, dass alle benötigten Bibliotheken installiert sind, indem Sie im folgende Zelle ausführen:<br>

In [ ]:
!pip install -r requirements.txt

## 4 Code

### 4.1 Daten Generierung

Zuerst wird die Bibliothek `Numpy` importiert, welche fundamentale Funktionalitäten für numerische Berechnungen in Python bereitstellt.

Anschließend werden Beispieldaten generiert. Dafür wird die vorgefertigte Funktion `generate_linear_data` aus der Datei `utilities/data.py` importiert und aufgerufen. Diese generiert Daten, die einer linearen Funktion $y=m\cdot x+n$ folgen, wobei `m` der Anstieg, `n` die y-Verschiebung und `num_samples` die Anzahl der generierten Datenpunkte ist.

Die vorgefertigten Funktionen `plot_data_points` und `plot_series` werden aus der Datei `utilities/visualization.py` importiert. Diese dienen der Visualisierung der Daten.

In [ ]:
import numpy as np                                                  # Numpy is a fundamental package for numerical computations in Python
from utilities.data import generate_linear_data                     # Custom module for generating linear data
from utilities.visualization import plot_data_points, plot_series   # Custom module for visualizing 2D 

X, Y = generate_linear_data(
    m           = 1,
    n           = 0,
    num_samples = 20
)
plot_data_points(X, Y)

Der Parameter `noise` fügt zufällig künstlich generiertes Rauschen zu den Daten hinzu. Da reale Daten meist nicht perfekt sind, wird dieses Rauschen hier durch Zufallswerte simuliert:

In [ ]:
X, Y = generate_linear_data(
    m           = 1.0,
    n           = 0.0,
    num_samples = 50,
    noise       = 0.15
)
plot_data_points(X, Y)

### 4.2 Lineare Regression

Das Ziel neuronaler Netze ist es, eine Funktion zu finden, die die Daten möglichst gut repräsentiert. Wir starten hier eine einfache lineare Abhängigkeit durch eine lineare Regression zu approximieren. Das ist genau das, was in einem einzelnen Neuron passiert.

#### 4.2.1 Daten generieren

Zuerst wird der Datensatz generiert $D=(X,Y)$:

&nbsp;&nbsp;&nbsp;$X$ ...Eingabedaten<br>
&nbsp;&nbsp;&nbsp;$Y$ ...Zieldaten $\to$ es soll gelernt werden, diese Daten in Abhängigkeit von $X$ vorherzusagen

In [ ]:
m           = 0.7   # Anstieg der Geraden
n           = 1.5   # y-Verschiebung der Geraden
noise       = 0.1   # Rauschanteil

X, Y = generate_linear_data(
    num_samples     = 50,
    m               = m,
    n               = n,
    noise           = noise
)

plot_data_points(input=X, target=Y, prediction=None, title=f"Linear Data with noise")

#### 4.2.2 Stochastic Gradient Descent (SGD)

Nun versuchen wir die Funktion $\hat{y}=w\cdot x+b$ so zu optimieren, dass sie die Daten möglichst gut abbildet. Dafür werden mittels **Gradient Descent Algorithmus** die freien Parameter $w$ und $b$ iterativ angepasst.

&nbsp;&nbsp;&nbsp;$x$ ...x-Wert aus den Daten (`inputs`)<br>
&nbsp;&nbsp;&nbsp;$y$ ...y-Wert aus den Daten (`targets`)<br>
&nbsp;&nbsp;&nbsp;$w$ ...weight (zu lernender Anstieg, unbekannt)<br>
&nbsp;&nbsp;&nbsp;$b$ ...bias (zu lernende y-Verschiebung, unbekannt)

> ---
> **Gradient Descent Algorithmus**
> 
> ---
>
> **Eingaben:**
> - Lernrate $\eta$ (eta)
> - Trainingsdaten $D = {(X,Y)}$
> - Anzahl Epochen $E$
> - Initiales weight $w\leftarrow 0$
> - Initialer Bias $b\leftarrow 0$
>
> **Ausgaben**
> - Vorhersage $Y$
> 
> **for** $epoch = 1,\,...,\,E$ **do**<br>
>> **for** $x_i,y_i\in D$ **do**<br>
>>> $\hat{y_i} = w\cdot x_i + b$<br>
>>> $dw = 2 \cdot (y_i-\hat{y_i}) \cdot x_i$<br>
>>> $db = 2 \cdot (y_i-\hat{y_i})$
>>>
>>> $w\leftarrow w - \eta \cdot dw$<br>
>>> $b\leftarrow b - \eta \cdot db$
>>
>> **end for**
>
> **end for**
>
> ---

**Erklärung**

Die lineare Regression minimiert die Summe der quadrierten Abweichungen zwischen tatsächlichen Werten $\hat{y}$ und vorhergesagten Werten $y$, indem sie den quadratischen Fehler (Mean Square Error - MSE) minimiert.
$$L_{MSE}=(y-\hat{y})^2$$
$$L_{MSE}=(y-(wx+b))^2$$

Man startet mit zufälligen Parametern weight $w$ und bias $b$ und verbessert sie in kleinen Schritten, indem man die partielle Ableitung der Loss-Funktion nach $w$ und $b$ bildet und diese multipliziert mit der Lernrate $\eta$ auf $w$ bzw. $b$ addiert:
$$w \leftarrow w - \eta \frac{\partial Loss}{\partial w}$$
$$b \leftarrow b - \eta \frac{\partial Loss}{\partial b}$$

Die partiellen Ableitungen der MSE-Loss-Funktion sind hierbei
$$\frac{\partial L_{MSE}}{\partial w}=2\cdot(wx+b-y)\cdot x=2\cdot(\hat{y}-y)\cdot x$$
$$\frac{\partial{L_{MSE}}}{\partial{b}}=2\cdot(wx+b-y)=2\cdot(\hat{y}-y)$$

In [ ]:
# Initialize parameters
EPOCHS      = 200       # Number of training epochs
LR          = 0.001     # Learning Rate
w, b        = 0., 0.    # weight and bias
losses      = []        # to store loss values

for epoch in range(EPOCHS):
    # prediction is a vector of target size, initialized with zeros
    Y_hat = np.zeros_like(Y)

    for i, (x, y) in enumerate(zip(X, Y)):
        y_hat = w * x + b  # Linear model prediction

        grad_w = 2 * (y_hat - y) * x
        grad_b = 2 * (y_hat - y)

        w -= LR * grad_w
        b -= LR * grad_b

        Y_hat[i] = y_hat

    loss = np.mean((Y_hat - Y) ** 2)
    losses.append(loss)

    if (epoch+1) % 20 == 0 or epoch == 0 or epoch == EPOCHS - 1:
        plot_data_points(input=X, target=Y, prediction=Y_hat, title=f"Linear Data Fitting - Epoch {epoch+1}")
        print(f"Epoch {epoch+1}/{EPOCHS}, MSE: {loss:0.6f}")

plot_series(data=losses, title="Training Loss over Epochs", xlabel="Epochs", ylabel="MSE Loss")
print(f'Learned parameters: w = {w.flatten()[0]:.4f}, b = {b.flatten()[0]:.4f}, True parameters: m = {m}, n = {n}')

Sie haben nun gelernt, wie ein einzelnes Neuron lernt. In Part 2 werden wir uns anschauen, wie dies in einem neuronalen Netz funktioniert.